In [ ]:
import telebot
from my_utils import config
from telebot import types
import os
import pathlib
import skimage
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt
from my_utils.wiki import WikiArticle
from my_utils import mashrooms as mr
from my_utils import preprocessing
import pandas as pd
import numpy as np
from pprint import pprint
import shutil

bot = telebot.TeleBot(config.APIKEY)
PHOTO_PATH = './photo_users'
os.makedirs(PHOTO_PATH, exist_ok=True)
# mashrooms_dict[0]


@bot.message_handler(commands=['edit'])
def edit(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("🇷🇺 Русский")
    btn2 = types.KeyboardButton('🇬🇧 English')
    markup.add(btn1, btn2)
    bot.send_message(
        message.from_user.id, 
        "🇷🇺 Выберите язык / 🇬🇧 Choose your language", reply_markup=markup)

#############################################################
#############################################################
@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    # btn1 = types.KeyboardButton("👋 Поздороваться")
    # markup.add(btn1)
    bot.send_message(
        message.from_user.id, 
        "👋 Привет! Я твой бот-помошник! Отправьте мне название гриба и я найду его значение на Wikipedia", 
        reply_markup=markup
        )
#############################################################
#############################################################
@bot.message_handler(commands=['predict'])
def predict(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Run")
    markup.add(btn1)
    bot.send_message(
        message.from_user.id, 
        "Begin predict your image", 
        reply_markup=markup)
#############################################################
#############################################################
# @bot.message_handler(content_types=['text'])
# def get_text_messages(message):

#     if message.text == '👋 Поздороваться':
#         markup = types.ReplyKeyboardMarkup(resize_keyboard=True) #создание новых кнопок
#         btn1 = types.KeyboardButton('Распознать❓')
#         # btn2 = types.KeyboardButton('Правила сайта')
#         # btn3 = types.KeyboardButton('Советы по оформлению публикации')
#         # markup.add(btn1, btn2, btn3)
#         markup.add(btn1)
#         bot.send_message(
#             message.from_user.id, 
#             # '❓ Задайте интересующий вопрос', 
#             'Отправляю...', 
            # reply_markup=markup) #ответ бота
    # elif message.text == 'Правила сайта':
    #     bot.send_message(
    #         message.from_user.id, 
    #         'Прочитать правила сайта вы можете по ' + '[ссылке](https://habr.com/ru/docs/help/rules/)', 
    #         parse_mode='Markdown')

    # elif message.text == 'Советы по оформлению публикации':
    #     bot.send_message(
    #         message.from_user.id, 
    #         'Подробно про советы по оформлению публикаций прочитать по ' + '[ссылке](https://habr.com/ru/docs/companies/design/)', 
    #         parse_mode='Markdown')
#############################################################
#############################################################
@bot.message_handler(content_types=['photo', 'text'])
def photo(message):
    msg_content_type = message.content_type
    print(f"Content type: {msg_content_type}")
    print(f"Content message caption: {message.caption}")

    print(f"User ID: {message.from_user.id}\n"
          f"User UserNick: {message.from_user.username}\n"
          f"User FirstName: {message.from_user.first_name}\n"
          f"User LastName: {message.from_user.last_name}\n")

    if msg_content_type == 'text':
        print(f'Get text from bot. TEXT=={message.text}')
        msg_check = preprocessing.check_question_handle(message.text)
        print(f"Answer from dict:\n{msg_check}")
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        if len(msg_check)==0:
            bot.send_message(
                chat_id=message.from_user.id,
                text="*Ничего не нашел :(*",
                parse_mode='Markdown',
            )
        {j: markup.add(types.KeyboardButton(i)) for j, i in enumerate(msg_check)}
        [bot.send_message(
            chat_id= message.from_user.id,
            text = f"Найдено по запросу:\n*{i}*",
            reply_markup=markup,
            parse_mode='Markdown',
            ) for i in msg_check]
        
        if len(msg_check)==1:
            answer_wiki = preprocessing.wiki_article_info(msg_check[0], language='ru', show_image=True)
            print(f"Type answer Wiki: {answer_wiki}")
            if isinstance(answer_wiki, list):
                txt = "\n".join((",".join(answer_wiki)).split(','))
                bot.send_message(
                    chat_id=message.from_user.id,
                    text= "*Уточните запрос:*\n" + txt,
                    parse_mode='Markdown',
                )
            else:
                title_article, url, answer_wiki_page = answer_wiki
                txt = "".join(f"{answer_wiki_page[list(answer_wiki_page.keys())[0]]}").strip("'")
                txt = ".\n".join(txt.split('. '))
                print(url)
                bot.send_message(
                    chat_id= message.from_user.id,
                    text = f"*Статус:*\n{preprocessing.check_status(message.text)}\n",
                    parse_mode='Markdown'
    
                )                
                bot.send_message(
                    chat_id= message.from_user.id,
                    text = f"*Название статьи:*\n{title_article[0]}\n",
                    parse_mode='Markdown'
    
                )
                bot.send_message(
                    chat_id= message.from_user.id,
                    text = '*Описание:*\n' + txt,
                    parse_mode='Markdown'
                )
    
                img_lst = ["./temp_img/"+ i for i in os.listdir("./temp_img/")]
                [bot.send_photo(
                    chat_id= message.from_user.id,
                    photo=open(img, 'rb'),
    
                ) for img in img_lst]
                
                bot.send_message(
                    chat_id= message.from_user.id,
                    text = str(url),
                    # parse_mode='Markdown'
    
                )
        shutil.rmtree("./temp_img", ignore_errors=True)
    if msg_content_type == 'photo':
        print('message.caption =', message.caption)
        # print('message.text =', message.text)
        # print('message.photo =', message.photo)
        fileID = message.photo[-1].file_id
        # print('fileID =', fileID)
        file_info = bot.get_file(fileID)
        # print('file.file_path =', file_info.file_path)
        print('photo UID =', file_info.file_unique_id)
        width = message.photo[-1].width
        height = message.photo[-1].height
        file_size = message.photo[-1].file_size
        # print('message.photo =', message.photo[-1])
        print('Width =', width)
        print('Height =', height)
        print('Size =', file_size)
        downloaded_file = bot.download_file(file_info.file_path)
        msg_cap = '' if message.caption is None else message.caption
        output_path = f"{PHOTO_PATH}/uid{str(message.from_user.id)}"
        os.makedirs(output_path, exist_ok=True)
        with open(f"{output_path}/{file_info.file_unique_id}_{msg_cap.replace(' ', '_')}.jpg", 'wb') as new_file:
            new_file.write(downloaded_file)
#############################################################
#############################################################
# photo = open('/tmp/photo.png', 'rb')
# bot.send_photo(chat_id, photo)
# bot.send_photo(chat_id, "FILEID")
bot.polling(none_stop=True, interval=0)

In [ ]:
# msg_questions = [i['rus'] for i in check_question('Белый гриб')]
# msg_questions[0]

In [ ]:
# answer_wiki = wiki_article_info(msg_questions[0])

In [ ]:
# bot.polling(none_stop=True, interval=0)

In [ ]:
# import skimage
# from skimage.transform import resize
# import matplotlib.pyplot as plt
# img = skimage.io.imread("./temp_img/img.jpg")
# H, W = img.shape[:2]
# scale_H = 256/H
# scale_W = 256/W
# scale_H, scale_W
# if H <= W:
#     img_re = resize(img, (int(H*scale_H), int(W*scale_H)))
# if H > W:
#     img_re = resize(img, (int(H*scale_W), int(W*scale_W)))
# plt.imshow(img_re)

In [ ]:
from my_utils.wiki import WikiArticle

page = WikiArticle('Рыжик сосновый', 'ru')
article = page.get_article_wiki()

In [ ]:
article.url

In [ ]:
import wikipedia as wiki
wiki.search('Чага')

In [ ]:
from my_utils import preprocessing
title, url, info_wiki = preprocessing.wiki_article_info('Рыжик сосновый', language='ru', show_image=False)
# title = answer['content'][0].split('\n')
# pprint(title)
# body = info_wiki['content'][1:]
# body_blocks_title = {j:i for j, i in enumerate(body) if i.startswith('==')}
# body_blocks_content = {j:i for j, i in enumerate(body) if not i.startswith('==')}
# A = np.array(list(body_blocks_title.keys()))
# B = np.array(list(body_blocks_content.keys()))
# body_content = []
# info_wiki = {}
# for i in range(len(A)+1):
#     try:
#         res = B[np.where((B[:] >= A[i]) & (B[:] < A[i+1]))].tolist()
#         body_content = [body_blocks_content[j] for j in res]
#         info_wiki[body_blocks_title[A[i]].strip().replace('=','').strip()] = body_content
#         body_content = []
#     except Exception as e:
#         continue
info_wiki

In [ ]:
list(info_wiki.keys())[0]